In [ ]:
import requests
import csv

url = "https://edamam-food-and-grocery-database.p.rapidapi.com/api/food-database/v2/parser"

querystring = {"ingr": "champagne"}

headers = {
    "X-RapidAPI-Key": "0637aead30msh30beae77deecd1cp147930jsn4fdf18ece750",
    "X-RapidAPI-Host": "edamam-food-and-grocery-database.p.rapidapi.com",
}

rows = []

def process_response(response):
    data = response.json()

    for item in data.get("hints", []):
        food_id = item.get("food", {}).get("foodId", "")
        label = item.get("food", {}).get("label", "")
        category = item.get("food", {}).get("category", "")
        contents_label = item.get("food", {}).get("foodContentsLabel", "")
        image = item.get("food", {}).get("image", "")

        rows.append([food_id, label, category, contents_label, image])

# Initial request
response = requests.get(url, headers=headers, params=querystring)

# Check if the request was successful (status code 200)
# page = 1
if response.status_code == 200:
    # print("page 1")
    process_response(response)

    # Check for additional pages
    while "_links" in response.json() and "next" in response.json()["_links"]:
        next_url = response.json()["_links"]["next"]["href"]
        response = requests.get(next_url, headers=headers)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # page = page + 1
            # print(f"page {page}")
            process_response(response)
        else:
            print("Error in fetching next page:", response.status_code)
            print(response.text)
            break

    # Save the data to a CSV file
    savedatafile = "output_API.csv"
    with open(savedatafile, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        # Write header
        writer.writerow(["foodId", "label", "category", "foodContentsLabel", "image"])
        # Write rows
        writer.writerows(rows)

    print(f"Data saved to {savedatafile}")
else:
    print("Error:", response.status_code)
    print(response.text)
